In [ ]:
# INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
# 
# Copyright © 2019-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

In [2]:
import nxsdk.api.n2a as nx
import time
import numpy as np

from nxsdk_modules.slayer.src.slayer2loihi import Slayer2Loihi as s2l
from nmnistDataset import NmnistDataset

import os
os.environ['SLURM'] = '1'
os.environ['PARTITION'] = 'nahuku32'

### Get the latest SLAYER models

In [3]:
modelPath = s2l.getModels() + '/02_NMNIST/'

### Get the NMNIST test data

In [4]:
dataset = NmnistDataset('/nfs/ncl/datasets/NMNIST_test')

In [5]:
spikesPerPacket = 2048

loadState = False
saveState = False
boardName = 'nmnist'

regenerateCoreAxon = not loadState

# How many Lakemonts to distribute spike injection across
numSnips = 2

# The NeuroCore from which we'll start placing compartments
corenum = 0 

# create an empty network
net = nx.NxNet()

compProto = s2l.compartmentPrototype(modelPath+'network.yaml')

## Input Layer
34x34x2

In [6]:
inputSpec = dict()
inputSpec["sizeX"] = 34
inputSpec["sizeY"] = 34
inputSpec["sizeC"] = 2
inputSpec["connProto"] = compProto

compartmentsPerCore = 350

layerInput, inputConnectionGroup, corenum = s2l.inputLayer(net, inputSpec, corenum, compartmentsPerCore)

# Switching from x-y-p co-ordinates to a fully connected layer requires re-ordering
layerInput = s2l.reorderLayer(layerInput)

## Layer 1
512 Fully Connected

In [7]:
fullSpec = dict()
fullSpec["compProto"] = compProto
fullSpec["dim"] = 512
fullSpec["weightFile"] = modelPath + '/Trained/NMNISTFc1.npy'

compartmentsPerCore = 64 

layer1, corenum  = s2l.fullLayer(layerInput, fullSpec, corenum, compartmentsPerCore)

## Output Layer
10 Fully Connected

In [8]:
fullSpec = dict()
fullSpec["compProto"] = compProto
fullSpec["dim"] = 10
fullSpec["weightFile"] = modelPath + '/Trained/NMNISTFc2.npy'

compartmentsPerCore = 10

layerOutput, corenum  = s2l.fullLayer(layer1, fullSpec, corenum, compartmentsPerCore)

In [9]:
dummyProbes = s2l.setupSpikeCounters(layerOutput)

In [10]:
if loadState is False:
    compiler = nx.N2Compiler()
    board = compiler.compile(net)
else:
    board, dummyProbes = s2l.initBoard(boardName)

In [11]:
numSamples = len(dataset)

blankTime = 50 #how many blank timesteps gap between samples
spikeTime = dataset.sampleLength
sampleLength = spikeTime + blankTime

s2l.writeHeader(layerOutput, spikesPerPacket, sampleLength)
spikeChannels, core, axon = s2l.prepSpikeInjection(inputConnectionGroup, board, spikesPerPacket, sampleLength, numSnips, regenerateCoreAxon)
spikeData, numSteps = s2l.prepSpikeData(core, axon, spikesPerPacket, layerInput, dataset, numSamples, sampleLength, numSnips)

In [12]:
numClasses = layerOutput.numNodes
spikeCntrChannel = s2l.prepSpikeCounter(board, numSamples, numClasses, int(corenum))

In [13]:
board.start()
if saveState is True:
    s2l.saveBoard(board, boardName, dummyProbes)
if loadState is True:
    s2l.loadBoard(board, boardName)
board.run(numSteps, aSync=True)

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.92:38283
INFO:DRV:      Host server up..............Done 0.47s
INFO:DRV:      Encoding axons/synapses.....Done 1.65s
INFO:DRV:      Compiling Embedded snips....Done 0.91s
INFO:DRV:      Compiling MPDS Registers....Done 0.63ms
INFO:HST:  Config /etc/nx/pio.cfg
INFO:HST:  Args chip=0 cpu=0 /home/sshresth/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1612580162.8726735/launcher_chip0_lmt0.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=0 cpu=1 /home/sshresth/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1612580162.8726735/launcher_chip0_lmt1.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 0.84s
INFO:DRV:      Encoding probes.............Done 0.62ms
INFO:DRV:      Transferring probes.........Done 3.80ms
INFO:DRV:      Configuring registers.......Done 0.95s
INFO:DRV:      Transferring spikes.........Done 0.53ms


In [14]:
tStart = time.time()
s2l.sendSpikeData(spikeData, spikeChannels, spikesPerPacket)

In [15]:
numClasses = layerOutput.numNodes
results = s2l.getResults(spikeCntrChannel, numSamples, numClasses, dummyProbes)

In [16]:
board.finishRun()
board.disconnect()
tEnd = time.time()
print("Completed {} in  {:.2f} seconds".format(numSteps, tEnd-tStart))

INFO:DRV:      Executing...................Done 2.30ms
INFO:DRV:      Processing timeseries.......Done 2.35ms
INFO:HST:  chip=0 cpu=0 halted, status=0x0
INFO:HST:  chip=0 cpu=1 halted, status=0x0
Completed 4001000 in  124.66 seconds


In [17]:
labels = dataset.labels
accuracy = s2l.checkAccuracy(labels, results)
print("Accuracy is {:.2f}%".format(accuracy*100))

Accuracy is 98.46%
